In [188]:
import json
import math
import pandas as pd

What we're aiming for is something like this

```json
{
  "E14999999":{
    "name": "Fake news",
    "code": "E14999999",
    "description": "This constituency is great!",
    "headlines": [
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"}
    ],
		"similar": { "People": 4 },
		"leastsimilar": { "People": 1000 },
		"neet": {
			"something": [34,56,24,19,25],
			"another": {}
		},
		"employment": {
			
		}
	}
}
```

In [189]:
descriptions=pd.read_csv('../../data/reference/constituency-descriptions.csv', index_col='PCON22CD').fillna('')

In [190]:
pcons=pd.read_json('../../src/_data/areas/reference/pcon.json')

In [191]:
summary=pd.merge(pcons, descriptions, how='left', left_on='PCON22CD', right_index=True).set_index('PCON22CD')

summary.rename(columns={ 'PCON22NM': 'name'}, inplace=True)
summary.rename(columns=lambda x: x.lower(), inplace=True)

In [192]:
summary = summary.to_dict(orient='index')

In [193]:
employment_headlines = pd.read_csv('../../data/area/pcon/headlines.csv', index_col='PCON22CD')
education_headlines = pd.read_csv('../../data/area/pcon/education_attainment_pcon_2010.csv', index_col='PCON22CD')


econ_inactive = pd.read_csv('../../data/area/pcon/econ_inactive_16-24_last_3_years.csv', index_col='PCON22CD')
unemployment = pd.read_csv('../../data/area/pcon/youth_unem_16-24_last_3_years.csv', index_col='PCON22CD')
economic_activity_16_19 = pd.read_csv('../../data/area/pcon/econ_active_16-19_last_3_years.csv', index_col='PCON22CD')
economic_activity_20_24 = pd.read_csv('../../data/area/pcon/econ_active_20-24_last_3_years.csv', index_col='PCON22CD')

In [197]:
economic_activity_20_24

,2021-09-01,2021-12-01,2022-03-01,2022-06-01,2022-09-01,2022-12-01,2023-03-01,2023-06-01,2023-09-01
PCON22CD,,,,,,,,,
E14000530,100.0,100.0,100.0,0.0,100.0,88.1,91.8,92.8,91.5
E14000531,83.8,86.5,86.5,80.8,86.7,83.5,71.2,90.6,86.6
E14000532,57.5,74.4,56.7,50.0,64.0,52.5,69.6,62.7,63.3
E14000533,73.2,72.8,89.3,84.8,82.0,77.8,0.0,0.0,100.0
E14000534,86.7,84.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
E14001058,66.5,56.7,0.0,56.0,77.2,0.0,74.2,78.5,81.2
E14001059,74.7,69.8,62.7,85.4,87.0,86.5,79.0,86.2,87.5
E14001060,54.9,78.7,83.0,80.7,100.0,100.0,100.0,100.0,0.0


In [195]:
for s in summary:
    suffix = employment_headlines.loc[s, :].Suffix
    summary[s]['employment_headlines'] = [
        { 'h': headline_name, 'v': headline_value, 'suffix': suffix }
        for headline_name, headline_value
        in employment_headlines.loc[s, :].drop(labels="Suffix").items()
        ]
    summary[s]['education_headlines'] = [
        { 'h': name, 'v': value, 'suffix': suffix}
        for name, value
        in education_headlines.loc[s, :].drop(labels="suffix").items()
        ]
    try: 
        summary[s]['economic_inactivity'] = [
            { 'year': date_name, 'value': value }
            for date_name, value
            in econ_inactive.loc[s,:].items()
        ]
    except: 
        print(f'No data found for constituency: {s}')
    try: 
        summary[s]['economic_activity_16_19'] = [
            { 'year': date_name_16_19, 'value': value_16_19 }
            for date_name_16_19, value_16_19
            in economic_activity_16_19.loc[s,:].items()
        ]
    except: 
        print(f'No data found for constituency: {s}')
    try:
        summary[s]['economic_activity_20_24'] = [
            { 'year': date_name_20_24, 'value': value_20_24 }
            for date_name_20_24, value_20_24
            in economic_activity_20_24.loc[s,:].items()
        ]
    except: 
        print(f'No data found for constituency: {s}')
    
    '''Do this for all other things...'''

In [196]:
with open('../../src/generated/areas/constituency/_data/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)